In [2]:
import tensorflow as tf
import numpy as np

In [3]:
# generates random samples with some logic encoded in the values
def generator(num_samples):
  sample_data = np.random.randint(0, high=100, size=(num_samples))
  sample_labels = []
  for value in sample_data:
    sample_labels.append(1 if (value >= 50) else 0)
  print("Generating data. If number in sample is greater than 50, the label it's a 1")
  print(sample_labels[:10])
  print(sample_data[:10])
  return sample_data, sample_labels

def input_fn(dataset):
  def parser(feature, label):
    # do custom parsing, read files, transform, etc
    return {"x": feature}, label
  
  d = dataset.map(parser)
  d = d.batch(10)
  d = d.repeat(100)
  d = d.make_one_shot_iterator()
  features, label = d.get_next()
  return features, label

def train_input_fn():
  data, labels = generator(num_samples=2000)
  dataset = tf.data.Dataset.from_tensor_slices((data, labels))
  return input_fn(dataset)

def evaluate_input_fn():
  data, labels = generator(num_samples=100)
  dataset = tf.data.Dataset.from_tensor_slices((data, labels))
  return input_fn(dataset)

# model
features = [
    tf.feature_column.numeric_column("x")
]
estimator = tf.estimator.DNNClassifier(feature_columns=features, hidden_units=[32])

# train
estimator.train(input_fn=train_input_fn, steps=5000)

# evaluate
estimator.evaluate(input_fn=evaluate_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_num_worker_replicas': 1, '_session_config': None, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10ce6b390>, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_evaluation_master': '', '_save_checkpoints_secs': 600, '_service': None, '_master': '', '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_save_checkpoints_steps': None, '_model_dir': '/var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpzty457v3', '_is_chief': True}
Generating data. If number in sample is greater than 50, the label it's a 1
[1, 0, 1, 0, 0, 0, 0, 0, 1, 1]
[69 39 59 45 18 17 32 42 76 79]
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_ini

{'accuracy': 0.98,
 'accuracy_baseline': 0.53,
 'auc': 1.0,
 'auc_precision_recall': 1.0,
 'average_loss': 0.04792924,
 'global_step': 5000,
 'label/mean': 0.47,
 'loss': 0.4792924,
 'prediction/mean': 0.48578537}